# 1. <a id='toc1_'></a>[Preprocess](#toc0_)

This notebook contains the instructions for preprocessing the data. The preprocessing steps are as follows:

**Table of contents**<a id='toc0_'></a>    
1. [Preprocess](#toc1_)    
1.1. [Dependencies and paths](#toc1_1_)    
1.2. [Load the pickled data](#toc1_2_)    
1.3. [Check which pickled files are corrupted](#toc1_3_)    
1.4. [Split the data into train, validate and test sets based on the seasonality](#toc1_4_)    
1.5. [(intermediate-) Save the train, validate and test ``Dataset`` objects as pickled files](#toc1_5_)    
1.6. [Extend the feature set with some temoporal features](#toc1_6_)    
1.7. [(intermediate-) Save the extended train, validate and test ``pd.DataFrames`` objects as pickled files](#toc1_7_)    
1.8. [Explore the train, validate and test sets](#toc1_8_)    
1.9. [Remove Stops, Clean all NaN rows, and Downscale the datasets](#toc1_9_)    
1.10. [Scale the train and validate sets. Save the scaler as a pickled file](#toc1_10_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1.1. <a id='toc1_1_'></a>[Dependencies and paths](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## DEPENDENCIES >>>
import os
import sys
from pathlib import Path

# Add root directory to path for imports >
root_dir = Path.cwd().resolve().parent
if root_dir.exists():
    sys.path.append(str(root_dir))
else:
    raise FileNotFoundError('Root directory not found')

# import custom libraries >
from src.load import load_multiple_trajectoryCollection_parallel_pickle as lmtp
from src.load import load_df_to_dataset
from src.traj_dataloader import (separate_files_by_season, 
                                 split_data, 
                                 get_files,
                                 AISDataset,
                                 )

import dotsi
import pickle

import numpy as np

# Plot >
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots  # https://github.com/garrettj403/SciencePlots?tab=readme-ov-file
plt.style.use(['science', 'grid', 'notebook'])  # , 'ieee'

# %matplotlib inline
%matplotlib widget

In [3]:
## FLAGS & GLOBAL VALUES >>>
# Key for lines used for exploration and not for execution
EXPLORE = False

# Key for debugging
DEBUG = False

# Key if features are not divided into train, validate and test and saved as pickles
SPLIT_TVT_DATA = False

# Extend TVT data with temporal featues
EXTEND_TVT_DATA = False

# Remove Anchoring Ships
REMOVE_STOP = False

# Down sample the resolution
DOWN_SAMPLE = True  # used with SCALE and SAVE_SCALE to save the scaled data: (if True) with down sampled resolution, or with (not False) not.

# Drop corrupted features  # See corrupted_features list
DROP_CORRUPTED_FEATURES = True

# Scale TVT data
SCALE = True

# seed
split_seed = 42

# If DOWN_SAMPLE, define the target time resolution
targeted_resolution_min = 1  # minute

# TODO: The following featues are corrupted by containing NaNs. Fix this. For now, these columns are dropped
corrupted_features = ["stopped", "abs_ccs", "curv"]

In [4]:
## PATHS >>>
# data dir
data_dir = root_dir / 'data'
data_dir = data_dir.resolve()
if not data_dir.exists():
    raise FileNotFoundError('Data directory not found')

# aistraj dir
assets_dir = root_dir / 'data' / 'local' / 'aistraj'
assets_dir = assets_dir.resolve()
if not assets_dir.exists():
    raise FileNotFoundError('Assets directory not found')

# models dir
models_dir = root_dir / 'models'
models_dir = models_dir.resolve()
if not models_dir.exists():
    raise FileNotFoundError('Models directory not found')

# raw-pickle dir
raw_pickle_dir = assets_dir / 'pickle'
raw_pickle_dir = raw_pickle_dir.resolve()
if not raw_pickle_dir.exists():
    raise FileNotFoundError('Pickle Assets directory not found')

# train-validate-test (tvt) dir
tvt_assets_dir = assets_dir / 'tvt_assets'
tvt_assets_dir = tvt_assets_dir.resolve()
if not tvt_assets_dir.exists():
    raise FileNotFoundError('Train-Validate-Test Assets directory not found')

# tvt: original pickle dir
tvt_original_dir = tvt_assets_dir / 'original'
tvt_original_dir = tvt_original_dir.resolve()
if not tvt_original_dir.exists():
    raise FileNotFoundError('TVT Original Pickled Data directory not found')

# tvt: extended pickle dir
tvt_extended_dir = tvt_assets_dir / 'extended'
tvt_extended_dir = tvt_extended_dir.resolve()
if not tvt_extended_dir.exists():
    raise FileNotFoundError('TVT Extended Pickled Data directory not found')

# tvt: scaled pickle dir
tvt_scaled_dir = tvt_assets_dir / 'scaled'
tvt_scaled_dir = tvt_scaled_dir.resolve()
if not tvt_scaled_dir.exists():
    raise FileNotFoundError('TVT Scaled Pickled Data directory not found')

# tvt: logs dir
tvt_logs_dir = tvt_assets_dir / 'logs'
tvt_logs_dir = tvt_logs_dir.resolve()
if not tvt_logs_dir.exists():
    raise FileNotFoundError('TVT logs directory not found')

## 1.2. <a id='toc1_2_'></a>[Load the pickled data](#toc0_)

+ Get list of the pickle files

In [ ]:
if SPLIT_TVT_DATA:
    pickle_files_lst = get_files(str(raw_pickle_dir), extension='.pickle')  # get_files(str(test_pickle_assets_dir), extension='.pickle')
    print(pickle_files_lst)
    print(len(pickle_files_lst))

+ Get the absolute path and ckeck if the absolute path is recognised

In [ ]:
if SPLIT_TVT_DATA:
    if pickle_files_lst:
        absolute_paths = [os.path.join(raw_pickle_dir, file) for file in pickle_files_lst]
        existing_files = [file for file in absolute_paths if os.path.exists(file)]
        print("Number of Existing files:", len(existing_files))
        non_existing_files = [file for file in absolute_paths if not os.path.exists(file)]
        print("Existing files:", existing_files)
        print("Non-existing files:", non_existing_files)
    else:
        print("pickle_files_lst is empty") 

## 1.3. <a id='toc1_3_'></a>[Check which pickled files are corrupted](#toc0_)

As one pickle was corrupted and it broke the loop, I will check which pickle files are corrupted and remove them from the list of pickles to load.

In [ ]:
#> As one pickle is corrupted, we need to load the pickle files and check if they are valid
if DEBUG:
    from src.load import load_multiple_trajectoryCollection_pickle as lp
    for file in pickle_files_lst:
        print(f'loading {file}: ', end=' ')
        try:
            abs_file = os.path.join(raw_pickle_dir, file)
            df = lp(abs_file)
            print('success')
            del df
        except Exception as e:
            print('failed:', e)

## 1.4. <a id='toc1_4_'></a>[Split the data into train, validate and test sets based on the seasonality](#toc0_)

+ Split the file names based on the seasonality

In [ ]:
if SPLIT_TVT_DATA:
    summer_files, autumn_files, winter_files, spring_files = separate_files_by_season(pickle_files_lst)
    all_files_by_season = {'summer': summer_files, 'autumn': autumn_files, 'winter': winter_files, 'spring': spring_files}

    for season_key, season_values in all_files_by_season.items():
        print(season_key, len(season_values))
        if season_values:
            print(season_values)  
        print("=="*20)

+ Create the files list for train, validate and test. For each season, split the data to train (64%), validate (16%) and test (20%) sets

In [ ]:
if SPLIT_TVT_DATA:
    train_files_lst, validate_files_lst, test_files_lst = [], [], []

    for season_key, season_values in all_files_by_season.items():
        if season_values:  # If the list is not empty
            train_lst, validate_lst, test_lst = split_data(season_values, 
                                                           test_percent=0.2, 
                                                           validate_percent=0.2, 
                                                           random_seed=split_seed)
            train_files_lst.extend(train_lst)
            validate_files_lst.extend(validate_lst)
            test_files_lst.extend(test_lst)
            
            #> print for debug
            print(season_key, len(season_values))
            print("Train files:", len(train_lst))
            print("Validate files:", len(validate_lst))
            print("Test files:", len(test_lst))
            print("=="*20)
        else:  # List is empty
            print(f"{season_key} is empty")
            print("=="*20)
        
    print("len train files:", len(train_files_lst))
    print("len validate files:", len(validate_files_lst))
    print("len test files:", len(test_files_lst))

+ Create the ``torch.utils.data.Dataset`` objects for each set

In [ ]:
# Validation Dataset object
if SPLIT_TVT_DATA:
    validate_dataset = AISDataset(validate_files_lst, 
                                  raw_pickle_dir, 
                                  transform=None, 
                                  num_workers=16)
    print(len(validate_dataset))

In [ ]:
# Test Dataset object
if SPLIT_TVT_DATA:
    test_dataset = AISDataset(test_files_lst, 
                              raw_pickle_dir, 
                              transform=None, 
                              num_workers=16)
    print(len(test_dataset))

In [ ]:
# Train Dataset object
if SPLIT_TVT_DATA:
    train_dataset = AISDataset(train_files_lst, 
                               raw_pickle_dir,
                               transform=None, 
                               num_workers=16)
    print(len(train_dataset))

## 1.5. <a id='toc1_5_'></a>[(intermediate-) Save the train, validate and test ``Dataset`` objects as pickled files](#toc0_)

+ Save the ``Dataset`` objects as pickled files in ``tvt_original_dir``

In [ ]:
# Save Train >
if SPLIT_TVT_DATA:
    with open(tvt_original_dir / 'train_dataset.pickle', 'wb') as f:
        pickle.dump(train_dataset, f)

In [ ]:
# Save Test >
if SPLIT_TVT_DATA:
    with open(tvt_original_dir / 'test_dataset.pickle', 'wb') as f:
        pickle.dump(test_dataset, f)

In [ ]:
# Save Train >
if SPLIT_TVT_DATA:
    with open(tvt_original_dir / 'validate_dataset.pickle', 'wb') as f:
        pickle.dump(validate_dataset, f)

+ For logging and repeatedity in experiments. Save the train, validate, and test file names as txt

In [ ]:
if SPLIT_TVT_DATA:
    dataset = dotsi.Dict({'train': train_files_lst, 
                       'validate': validate_files_lst, 
                       'test': test_files_lst})
    
    for set in dataset:
        files_txt = "\n".join(dataset[set])
        files_txt = files_txt.replace(" ", "\n")  # Replace spaces with new lines
        with open(tvt_logs_dir / f'{set}_files.txt', 'w') as f:
            f.write(files_txt)

## 1.6. <a id='toc1_6_'></a>[Extend the feature set with some temoporal features](#toc0_)

+ Define a function to get the season from the date

In [ ]:
def assign_season(date):
    summer_start_date = 620
    autumn_start_date = 922
    winter_start_date = 1221
    spring_start_date = 320
    
    file_month_day = int(date.strftime('%m%d'))
    
    if spring_start_date <= file_month_day < summer_start_date:
        return 0  # Spring
    elif summer_start_date <= file_month_day < autumn_start_date:
        return 1  # Summer
    elif autumn_start_date <= file_month_day < winter_start_date:
        return 2  # Autumn
    else:
        return 3  # Winter

+ Add the temporal features

In [ ]:
if EXTEND_TVT_DATA:
    for df in [train_df, validate_df, test_df]:
        df['season'] = df['datetime'].apply(assign_season)
        # Assign part of the day based on hour using vectorized operations
        df['part_of_day'] = np.where((df['datetime'].dt.hour >= 6) & (df['datetime'].dt.hour <= 12), 0, 
                        np.where((df['datetime'].dt.hour >= 13) & (df['datetime'].dt.hour <= 18), 1, 2))
        # Encode hours, minutes, and seconds using sine and cosine to capture cyclical nature
        df['month_sin'] = np.sin(df['datetime'].dt.month * (2. * np.pi / 12))
        df['month_cos'] = np.cos(df['datetime'].dt.month * (2. * np.pi / 12))
        df['hour_sin'] = np.sin(df['datetime'].dt.hour * (2. * np.pi / 24))
        df['hour_cos'] = np.cos(df['datetime'].dt.hour * (2. * np.pi / 24))

## 1.7. <a id='toc1_7_'></a>[(intermediate-) Save the extended train, validate and test ``pd.DataFrames`` objects as pickled files](#toc0_)

In [ ]:
# Save Train df >
if EXTEND_TVT_DATA:
    with open(tvt_extended_dir / 'extend_train_df.pickle', 'wb') as f:
        pickle.dump(train_df, f)

In [ ]:
# Save Validate df >
if EXTEND_TVT_DATA:
    with open(tvt_extended_dir / 'extend_validate_df.pickle', 'wb') as f:
        pickle.dump(validate_df, f)

In [ ]:
# Save Test df >
if EXTEND_TVT_DATA:
    with open(tvt_extended_dir / 'extend_test_df.pickle', 'wb') as f:
        pickle.dump(test_df, f)

## 1.8. <a id='toc1_8_'></a>[Explore the train, validate and test sets](#toc0_)

In [ ]:
if EXPLORE:
    train_df = dataset.train.data
    validate_df = dataset.validate.data
    test_df = dataset.test.data

+ Train

In [ ]:
if EXPLORE:
    train_df.sample(3)

In [ ]:
if EXPLORE:
    train_df.describe()

In [ ]:
if EXPLORE:
    train_df.isna().sum()

+ Validate

In [ ]:
if EXPLORE:
    validate_df.sample(3)

In [ ]:
if EXPLORE:
    validate_df.describe()

In [ ]:
if EXPLORE:
    validate_df.isna().sum()

+ Test

In [ ]:
if EXPLORE:
    test_df.sample(3)

In [ ]:
if EXPLORE:
    test_df.describe()

In [ ]:
if EXPLORE:
    test_df.isna().sum()

> **NOTE**: </br> The following columns has lot of corrupted values (stored as NaN). Therefore, before any further processing, drop (using ``*_df.drop(['col1', 'col2'], axis=1)``) these columns from the data:
> + ``stopped``
> + ``abs_ccs``
> + ``curv``

## 1.9. <a id='toc1_9_'></a>[Remove Stops, Clean all NaN rows, and Downscale the datasets](#toc0_)

In [ ]:
target_paths = {
                'train': tvt_extended_dir / 'extend_train_df.pickle',
                'validate': tvt_extended_dir / 'extend_validate_df.pickle',
                'test': tvt_extended_dir / 'extend_test_df.pickle'
                }

In [ ]:
%%time
for k, v in target_paths.items():
    # Load the dataset >
    asset_df = pd.DataFrame()
    print(f"Loading {k} dataset")
    asset_df = load_df_to_dataset(v).data
    original_sample_cnt = asset_df.shape[0]
    print(f'\t\t>> sample count: {original_sample_cnt}')
    
    # Drop the NaN values. Ignore datetime value >
    print(f"\t> Dropping row with all nan values")
    asset_df = asset_df.dropna(subset= asset_df.columns.difference(['datetime', 'epoch']), how='all')
    new_sample_cnt = asset_df.shape[0]
    print(f'\t\t>> number of dropped samples due to rows of nan: {original_sample_cnt - new_sample_cnt}')
    last_sample_cnt = new_sample_cnt
        
    if DEBUG:  # Check if there are any NaN values
        print(f'\t\t>> counting NaN values in each column')
        nan_counts = asset_df.isna().sum()
        for column, value in nan_counts.items():
            print(f'\t\t\t>>> {column}: {value}')
    
    if REMOVE_STOP:  # Drop the stopped samples >
        print(f"\t> Dropping stopped samples")
        asset_df = asset_df[asset_df['stopped'] != 1]
        new_sample_cnt = asset_df.shape[0]
        print(f'\t\t>> number of dropped samples due to stopping: {last_sample_cnt - new_sample_cnt}')
        last_sample_cnt = new_sample_cnt
        
        if DEBUG:  # Check if there are any NaN values
            print(f'\t\t>> counting NaN values in each column')
            nan_counts = asset_df.isna().sum()
            for column, value in nan_counts.items():
                print(f'\t\t\t>>> {column}: {value}')
            
    if DOWN_SAMPLE:  # Reduce the resolution >
        print(f"\t> Downsampling for the resolution of {targeted_resolution_min} min")
        asset_df = reduce_resolution(asset_df, resolution_minutes=targeted_resolution_min)
        
        # TODO: For some reason,t he result df contains alot of missing values rows. This needs to be fixed. For now it is dropped
        asset_df = asset_df.dropna(subset= asset_df.columns.difference(['datetime', 'epoch']), how='all')
        
        new_sample_cnt = asset_df.shape[0]
        print(f'\t\t>> number of dropped samples due to downsampling the resolution for {targeted_resolution_min} min.: {last_sample_cnt - new_sample_cnt}')
        last_sample_cnt = new_sample_cnt
        
        if DEBUG:  # Check if there are any NaN values
            print(f'\t\t>> counting NaN values in each column')
            nan_counts = asset_df.isna().sum()
            for column, value in nan_counts.items():
                print(f'\t\t\t>>> {column}: {value}')
                
    print(f'\t> the final sample count: {asset_df.shape[0]}')
    print(f'\t\t>> counting NaN values in each column')
    nan_counts = asset_df.isna().sum()
    for column, value in nan_counts.items():
        print(f'\t\t\t>>> {column}: {value}')
    
    # Save the extended_cleaned df as parquet >
    print('Saving >>>', end=' ')
    if DOWN_SAMPLE:
        asset_df.to_parquet(tvt_extended_dir / f'cleaned_downsampled_extended_{k}_df.parquet')
    else:
        asset_df.to_parquet(tvt_extended_dir / f'cleaned_extended_{k}_df.parquet')
    print('success')
    del asset_df
    print('=='*20)

## 1.10. <a id='toc1_10_'></a>[Scale the train and validate sets. Save the scaler as a pickled file](#toc0_)

+ Load the cleaned datasets

In [5]:
import_paths = {'train': None, 'validate': None, 'test': None}

if DOWN_SAMPLE:
    import_paths = {
                    'train': tvt_extended_dir / 'cleaned_downsampled_extended_train_df.parquet',
                    'validate': tvt_extended_dir / 'cleaned_downsampled_extended_validate_df.parquet',
                    'test': tvt_extended_dir / 'cleaned_downsampled_extended_test_df.parquet',
                    }
else:
    import_paths = {
                    'train': tvt_extended_dir / 'cleaned_extended_train_df.parquet',
                    'validate': tvt_extended_dir / 'cleaned_extended_validate_df.parquet',
                    'test': tvt_extended_dir / 'cleaned_extended_test_df.parquet',
                    }

+ Declare the custom scaler

In [6]:
%%time

if SCALE:
    # Split the column names into groups for scaling each with different scaler >
    cols_not_to_scale = ['epoch', 'datetime', 'obj_id', 'traj_id', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos', 'season', 'part_of_day', 'stopped', 'abs_ccs', 'curv']
    cols_to_standard_scale = ['aad', 'cdd', 'dir_ccs'] 
    angles_to_min_max_scale = ['cog_c', 'rot_c'] # min=0, max=360
    distances_to_min_max_scale = ['distance_c', 'dist_ww', 'dist_ra', 'dist_cl', 'dist_ma']
    cols_to_robust_scale = ['speed_c', 'acc_c']
    location_min_max_scale = ['lon', 'lat']

    columns_in_order = cols_not_to_scale + cols_to_standard_scale + angles_to_min_max_scale + distances_to_min_max_scale + cols_to_robust_scale + location_min_max_scale

    # Define the scaler of each group >
    transformers = [('no_scaler', 'passthrough', cols_not_to_scale),
                    ('standard_scaler', StandardScaler(), cols_to_standard_scale),
                    ('angles_min_max_scaler', CustomMinMaxScaler(feature_range=(0,1), min=0, max=360), angles_to_min_max_scale),
                    ('distances_min_max_scaler', CustomMinMaxScaler(feature_range=(0,1), min=0), distances_to_min_max_scale),
                    ('robust_scaler', RobustScaler(), cols_to_robust_scale),
                    ('location_min_max_scale', MinMaxScaler(), location_min_max_scale)]

    # Create column transformer within a pipeline >
    scale_transformer =  ColumnTransformer(transformers, remainder='passthrough', n_jobs=10) # passthrough: None mentioned columns shall remain as it is

CPU times: user 30 µs, sys: 36 µs, total: 66 µs
Wall time: 76.3 µs


+ Load the datasets

In [7]:
%%time
if SCALE:
    # Fit the scaler on train and validate data >
    train_df = pd.read_parquet(import_paths['train'])  # Load the train dataset
        
    validate_df = pd.read_parquet(import_paths['validate'])  # Load the validate dataset

CPU times: user 904 ms, sys: 2.28 s, total: 3.19 s
Wall time: 2.87 s


In [10]:
train_df.head()

,epoch,stopped,cog_c,aad,rot_c,speed_c,distance_c,acc_c,cdd,abs_ccs,...,traj_id,lon,lat,obj_id,season,part_of_day,month_sin,month_cos,hour_sin,hour_cos
datetime,,,,,,,,,,,,,,,,,,,,,
2022-03-24 13:13:00,1648127580,0,99.080888,0.0,0.0,0.587292,0.0,0.0,0.0,0.0,...,0,10.139315,54.36597,209221000.0,0.0,1.0,1.0,6.123234e-17,-0.258819,-0.965926
2022-03-24 13:14:00,1648127640,0,104.065234,1.471039,0.147104,0.5063,5.063,-0.001443,33.386957,0.10741,...,0,10.139818,54.36591,209221000.0,0.0,1.0,1.0,6.123234e-17,-0.258819,-0.965926
2022-03-24 13:15:00,1648127700,0,95.703471,2.762937,-0.276294,0.201057,2.010571,-0.003743,53.728167,0.183571,...,0,10.140124,54.365873,209221000.0,0.0,1.0,1.0,6.123234e-17,-0.258819,-0.965926
2022-03-24 13:16:00,1648127760,0,163.765402,145.052777,14.505278,0.506937,5.069375,0.040647,65.10097,1.770759,...,0,10.140201,54.365856,209221000.0,0.0,1.0,1.0,6.123234e-17,-0.258819,-0.965926
2022-03-24 13:17:00,1648127820,0,89.999999,0.0,0.0,0.013426,0.134263,-0.000739,68.432622,2.399117,...,0,10.140222,54.365877,209221000.0,0.0,1.0,1.0,6.123234e-17,-0.258819,-0.965926


+ Concat the train and validate sets

In [12]:
%%time
if SCALE:
    combined_df = pd.concat([train_df, validate_df])  # Combine the train and validate datasets
    
    # For DOWN_SAMPLE: The datatime is used as index, therefore, it is reset to be a column >
    combined_df.reset_index(inplace=True)  # Reset index to move datetime index to a column and create a new numerical index

CPU times: user 72.4 ms, sys: 129 ms, total: 202 ms
Wall time: 200 ms


+ Fit the scaler on the train and validate sets (not the test!)

In [14]:
%%time
if SCALE:
    scale_transformer.fit(combined_df)  # Fit the scaler on the combined dataset
    
    # Delete the train, validate and combined dataset to free up the memory >
    del combined_df
    del train_df
    del validate_df

/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'abs_ccs' has dtype Float64 and uses pandas.NA to represent null values. Storing this output in a numpy array can cause errors in downstream scikit-learn estimators, and inefficiencies. Starting with scikit-learn version 1.6, this will raise a ValueError. To avoid this problem you can (i) store the output in a pandas DataFrame by using ColumnTransformer.set_output(transform='pandas') or (ii) modify the input data or the 'no_scaler' transformer to avoid the presence of pandas.NA (for example by using pandas.DataFrame.astype).
  warnings.warn(
/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'curv' has dtype Float64 and uses pandas.NA to represent null values. Storing this outpu

CPU times: user 6.22 s, sys: 4.13 s, total: 10.3 s
Wall time: 11.4 s


+ Save the fitted scaler in ``models_dir``:

In [15]:
%%time
if SCALE:
    # Define the scaler name >
    scaler_name = None
    if DOWN_SAMPLE:
        scaler_name = 'cleaned_downsampled_extended_scaler.pickle'
        column_order_name = 'cleaned_downsampled_columns_after_extended_scale_order.txt'
    else:
        scaler_name = 'cleaned_extended_scaler.pickle'
        column_order_name = 'cleaned_columns_after_scale_extended_order.txt'
    
    # Save the scaler >
    with open(models_dir / scaler_name, 'wb') as f:
        pickle.dump(scale_transformer, f)   
        
    # Save the columns order after scaling >
    with open(models_dir / column_order_name, 'w') as f:
        f.write("\n".join(columns_in_order))

CPU times: user 0 ns, sys: 1.95 ms, total: 1.95 ms
Wall time: 1.75 ms


+ Scale the train, validate and test sets. Then save

In [17]:
%%time
if SCALE:    
    for k, v in import_paths.items():
        # Load the dataset >
        asset_df = pd.DataFrame()
        print(f"Loading {k} dataset")
        asset_df = load_df_to_dataset(v).data
        
        if DOWN_SAMPLE:  # Reset the index to move datetime index to a column and create a new numerical index >
            asset_df.reset_index(inplace=True)  # Reset index to move datetime index to a column and create a new numerical index
        
        original_sample_cnt = asset_df.shape[0]
        print(f'\t\t>> sample count: {original_sample_cnt}')
        
        # Transform the dataset >
        print(f'\t> Transforming the {k} dataset')
        asset_df = pd.DataFrame(scale_transformer.transform(asset_df), columns=columns_in_order)
        print(f'\t\t>> transformation complete')
        
        if DROP_CORRUPTED_FEATURES:  # Drop the corrupted features >
            print(f"\t> Dropping corrupted features")
            asset_df = asset_df.drop(corrupted_features, axis=1)
            print(f'\t\t>> number of dropped corrupted features: {len(corrupted_features)}')
        
        # Save the scaled df as parquet >
        print('Saving >>>', end=' ')
        if DOWN_SAMPLE:
            asset_df.to_parquet(tvt_scaled_dir / f'scaled_cleaned_downsampled_extended_{k}_df.parquet')
        else:
            asset_df.to_parquet(tvt_scaled_dir / f'scaled_cleaned_extended_{k}_df.parquet')
        print('success')
        
        del asset_df
        print('=='*20)

Loading train dataset
		>> sample count: 3025515
	> Transforming the train dataset


/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'abs_ccs' has dtype Float64 and uses pandas.NA to represent null values. Storing this output in a numpy array can cause errors in downstream scikit-learn estimators, and inefficiencies. Starting with scikit-learn version 1.6, this will raise a ValueError. To avoid this problem you can (i) store the output in a pandas DataFrame by using ColumnTransformer.set_output(transform='pandas') or (ii) modify the input data or the 'no_scaler' transformer to avoid the presence of pandas.NA (for example by using pandas.DataFrame.astype).
  warnings.warn(
/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'curv' has dtype Float64 and uses pandas.NA to represent null values. Storing this outpu

		>> transformation complete
	> Dropping corrupted features
		>> number of dropped corrupted features: 3
Saving >>> success
Loading validate dataset
		>> sample count: 759664
	> Transforming the validate dataset


/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'abs_ccs' has dtype Float64 and uses pandas.NA to represent null values. Storing this output in a numpy array can cause errors in downstream scikit-learn estimators, and inefficiencies. Starting with scikit-learn version 1.6, this will raise a ValueError. To avoid this problem you can (i) store the output in a pandas DataFrame by using ColumnTransformer.set_output(transform='pandas') or (ii) modify the input data or the 'no_scaler' transformer to avoid the presence of pandas.NA (for example by using pandas.DataFrame.astype).
  warnings.warn(
/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'curv' has dtype Float64 and uses pandas.NA to represent null values. Storing this outpu

		>> transformation complete
	> Dropping corrupted features
		>> number of dropped corrupted features: 3
Saving >>> success
Loading test dataset
		>> sample count: 952965
	> Transforming the test dataset


/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'abs_ccs' has dtype Float64 and uses pandas.NA to represent null values. Storing this output in a numpy array can cause errors in downstream scikit-learn estimators, and inefficiencies. Starting with scikit-learn version 1.6, this will raise a ValueError. To avoid this problem you can (i) store the output in a pandas DataFrame by using ColumnTransformer.set_output(transform='pandas') or (ii) modify the input data or the 'no_scaler' transformer to avoid the presence of pandas.NA (for example by using pandas.DataFrame.astype).
  warnings.warn(
/home/gfalouji/miniconda3/envs/captn-nba/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:702: FutureWarning: The output of the 'no_scaler' transformer for column 'curv' has dtype Float64 and uses pandas.NA to represent null values. Storing this outpu

		>> transformation complete
	> Dropping corrupted features
		>> number of dropped corrupted features: 3
Saving >>> success
CPU times: user 28.8 s, sys: 11.4 s, total: 40.1 s
Wall time: 40.2 s


+ Check

In [18]:
if DOWN_SAMPLE:
    if load_df_to_dataset(import_paths['train']).data.shape[0] == load_df_to_dataset(tvt_scaled_dir / 'scaled_cleaned_downsampled_extended_train_df.parquet').data.shape[0]:
        print("all good!")
else:
    if load_df_to_dataset(import_paths['train']).data.shape[0] == load_df_to_dataset(tvt_scaled_dir / 'scaled_cleaned_extended_train_df.parquet').data.shape[0]:
        print("all good!")

all good!


In [20]:
if DOWN_SAMPLE:
    display(load_df_to_dataset(tvt_scaled_dir / 'scaled_cleaned_downsampled_extended_train_df.parquet').data.head())
else:
    display(load_df_to_dataset(tvt_scaled_dir / 'scaled_cleaned_extended_train_df.parquet').data.head())

,epoch,datetime,obj_id,traj_id,month_sin,month_cos,hour_sin,hour_cos,season,part_of_day,...,rot_c,distance_c,dist_ww,dist_ra,dist_cl,dist_ma,speed_c,acc_c,lon,lat
0,1648127580,2022-03-24 13:13:00,209221000.0,0,1.0,6.123234e-17,-0.258819,-0.965926,0.0,1.0,...,0.000000e+00,0.000000,0.005813,0.002512,0.006155,0.002512,-0.548516,0.012355,0.170229,0.069312
1,1648127640,2022-03-24 13:14:00,209221000.0,0,1.0,6.123234e-17,-0.258819,-0.965926,0.0,1.0,...,4.086218e-04,0.000383,0.005801,0.002498,0.006140,0.002498,-0.572914,-0.115195,0.170248,0.069310
2,1648127700,2022-03-24 13:15:00,209221000.0,0,1.0,6.123234e-17,-0.258819,-0.965926,0.0,1.0,...,-7.674825e-04,0.000152,0.005793,0.002489,0.006131,0.002489,-0.664865,-0.318543,0.170259,0.069308
3,1648127760,2022-03-24 13:16:00,209221000.0,0,1.0,6.123234e-17,-0.258819,-0.965926,0.0,1.0,...,4.029244e-02,0.000383,0.005791,0.002487,0.006129,0.002487,-0.572722,3.605529,0.170262,0.069308
4,1648127820,2022-03-24 13:17:00,209221000.0,0,1.0,6.123234e-17,-0.258819,-0.965926,0.0,1.0,...,1.002778e-10,0.000010,0.005790,0.002486,0.006128,0.002486,-0.721387,-0.052948,0.170263,0.069309
